In [ ]:
# Path configuration
import os

# Main project folder
BASE_PATH = "./data/"

# Folder containing thermal images
IMAGE_PATH = os.path.join(BASE_PATH, "thermal_images/")

# --- FOLDERS AUTOMATICALLY CREATED IN BASE_PATH ---

# Outut rom Feture etraction on New data
RESULT_FEATURES = os.path.join(BASE_PATH, "result_features/")

# Folder where the BRF model (random forest model) is stored
BRF_MODEL = os.path.join(BASE_PATH, "brf_model/")

# Folder where prediction results are saved
PREDICTIONS = os.path.join(BASE_PATH, "predictions/")

# Create the folders if they do not exist
for path in [BASE_PATH, RESULT_FEATURES, BRF_MODEL, IMAGE_PATH, PREDICTIONS]:
    os.makedirs(path, exist_ok=True)


## Predict

In [ ]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, average_precision_score, confusion_matrix, roc_curve, precision_recall_curve
import seaborn as sns
import os

output_file_path = os.path.join(PREDICTIONS, 'New_data_predictions.csv')

model_path = os.path.join(BRF_MODEL, 'BRF_model.sav')

# Load CSV with features
file_path = os.path.join(RESULT_FEATURES, 'image_features_elipse.csv')
csv_data = pd.read_csv(file_path, sep=';')

# Classical handcrafted features from CSV
csv_features = csv_data[['Major Axes', 'Minor Axes', 'Std PCA 1', 'Std PCA 2',
                         'Eccentricity', 'Circumference', 'Area',
                         'Median Difference', 'Max Difference', 'Min Difference',
                         'Remaining Temp Range', 'Ellipse Temp Range']]

# Load CNN activation maps
activations_layer_1 = np.load(RESULT_FEATURES + 'activations_layer_1_cutouts.npy')
activations_layer_2 = np.load(RESULT_FEATURES + 'activations_layer_2_cutouts.npy')
activations_layer_3 = np.load(RESULT_FEATURES + 'activations_layer_3_cutouts.npy')
activations_layer_4 = np.load(RESULT_FEATURES + 'activations_layer_4_cutouts.npy')

# Compute average activations per filter
def calculate_mean_activation(activations):
    return np.mean(activations, axis=(1, 2))

activations_layer_1_mean = calculate_mean_activation(activations_layer_1)
activations_layer_2_mean = calculate_mean_activation(activations_layer_2)
activations_layer_3_mean = calculate_mean_activation(activations_layer_3)
activations_layer_4_mean = calculate_mean_activation(activations_layer_4)

# Concatenate all features (CNN activations + handcrafted)
X = np.hstack([
    activations_layer_1_mean,
    activations_layer_2_mean,
    activations_layer_3_mean,
    activations_layer_4_mean,
    csv_features
])

# Load trained classifier
with open(model_path, 'rb') as model_file:
    model = pickle.load(model_file)

# Predict probabilities for class 1
probas = model.predict_proba(X)[:, 1]

# Apply classification threshold
threshold = 0.4
y_pred_threshold = (probas >= threshold).astype(int)

# Add prediction results to the original DataFrame
csv_data['predicted_proba'] = probas
csv_data['predicted_class'] = y_pred_threshold

# Display summary statistics
num_class_1 = np.sum(y_pred_threshold == 1)
num_class_0 = np.sum(y_pred_threshold == 0)

print(f"Number of objects classified as 1 (threshold={threshold}): {num_class_1}")
print(f"Number of objects classified as 0 (threshold={threshold}): {num_class_0}")

# Save prediction results to CSV
csv_data.to_csv(output_file_path, index=False, sep=';')
print(f"Prediction probabilities saved to {output_file_path}")


## Visualize Detected Objects

In [ ]:
import pandas as pd
import rioxarray as rxr
import matplotlib.pyplot as plt
import os
from matplotlib.patches import Rectangle
import warnings

warnings.filterwarnings("ignore")

predicted_file = os.path.join(PREDICTIONS, 'New_data_predictions.csv')

# Parameters
window_size = 30
show_class_0 = False  # False = show only class 1 (animals), True = show all

# Load prediction results
df = pd.read_csv(predicted_file, sep=';')

# Check required columns
required_columns = ['predicted_proba', 'predicted_class', 'cx', 'cy', 'img_name']
missing = [col for col in required_columns if col not in df.columns]
if missing:
    raise ValueError(f"Missing required columns: {missing}")

# Print class counts
print("Number of objects by 'predicted_class':")
print(f"   - Class 1 (animal): {(df['predicted_class'] == 1).sum()}")
print(f"   - Class 0 (other):  {(df['predicted_class'] == 0).sum()}")

# Visualization function
def visualize_segments(img_name, cx_cy_list, class_list, prob_list, idx_list):
    img_path = os.path.join(IMAGE_PATH, img_name)
    if not os.path.exists(img_path):
        print(f"Image not found: {img_path}")
        return

    img = rxr.open_rasterio(img_path).squeeze().values
    plt.figure(figsize=(6, 6))
    plt.imshow(img, cmap='magma')

    for (cx, cy), pred_class, proba, idx in zip(cx_cy_list, class_list, prob_list, idx_list):
        color = 'g' if pred_class == 1 else 'r'
        plt.plot(cx, cy, color + 'o', markersize=2)

        rect = Rectangle((cx - window_size // 2, cy - window_size // 2), window_size, window_size,
                         edgecolor=color, facecolor='none', linewidth=1)
        plt.gca().add_patch(rect)

        label = f"{proba:.2f}"
        plt.annotate(label,
                     (cx - window_size // 2 + 2, cy - window_size // 2 - 1),
                     color='white',
                     fontsize=6,
                     ha='left', va='bottom',
                     weight='bold',
                     bbox=dict(boxstyle="round,pad=0.2", facecolor="black", edgecolor="none", alpha=0.4))

    plt.title(f"Detections: {img_name}")
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# Loop through each image and visualize
for img_name, group in df.groupby('img_name'):
    if not show_class_0:
        group = group[group['predicted_class'] == 1]

    if group.empty:
        continue

    central_points = list(zip(group['cx'], group['cy']))
    classes = group['predicted_class'].tolist()
    probas = group['predicted_proba'].tolist()
    indices = group.index.tolist()

    visualize_segments(img_name, central_points, classes, probas, indices)
